In [1]:
import pandas as pd
import os

data_dir = r"C:\Users\Les\Downloads\coursework\data\raw"

files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

dataframes = []
for file in files:
    file_path = os.path.join(data_dir, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

print("Типы данных:")
print(data.dtypes)
print("\nРазмер данных (строки, столбцы):", data.shape)
print("\nПервые строки:")
print(data.head())
print("\nПоследние строки:")
print(data.tail())

Типы данных:
Site              object
Parameter         object
Date (LT)         object
Year               int64
Month              int64
Day                int64
Hour               int64
NowCast Conc.    float64
AQI                int64
AQI Category      object
Raw Conc.        float64
Conc. Unit        object
Duration          object
QC Name           object
dtype: object

Размер данных (строки, столбцы): (42855, 14)

Первые строки:
      Site          Parameter            Date (LT)  Year  Month  Day  Hour  \
0  Bishkek  PM2.5 - Principal  2019-02-06 06:00 AM  2019      2    6     6   
1  Bishkek  PM2.5 - Principal  2019-02-06 07:00 AM  2019      2    6     7   
2  Bishkek  PM2.5 - Principal  2019-02-06 08:00 AM  2019      2    6     8   
3  Bishkek  PM2.5 - Principal  2019-02-06 09:00 AM  2019      2    6     9   
4  Bishkek  PM2.5 - Principal  2019-02-06 10:00 AM  2019      2    6    10   

   NowCast Conc.  AQI AQI Category  Raw Conc. Conc. Unit Duration QC Name  
0         -999.0

In [2]:
def analyze_missing_and_invalid_values(data):
    missing_values = data.isnull().sum()
    print("\nПропущенные значения по столбцам:")
    print(missing_values)

    invalid_values = (data == -999).sum()
    print("\nКоличество некорректных значений (-999) по столбцам:")
    print(invalid_values)

    negative_values = (data.select_dtypes(include=['float64', 'int64']) < 0).sum()
    print("\nКоличество отрицательных значений по числовым столбцам:")
    print(negative_values)

    print("\nСводная статистика для числовых столбцов:")
    print(data.describe())

analyze_missing_and_invalid_values(data)



Пропущенные значения по столбцам:
Site               0
Parameter          0
Date (LT)          0
Year               0
Month              0
Day                0
Hour               0
NowCast Conc.      0
AQI                0
AQI Category     221
Raw Conc.          0
Conc. Unit         0
Duration           0
QC Name            0
dtype: int64

Количество некорректных значений (-999) по столбцам:
Site               0
Parameter          0
Date (LT)          0
Year               0
Month              0
Day                0
Hour               0
NowCast Conc.    221
AQI              221
AQI Category       0
Raw Conc.        132
Conc. Unit         0
Duration           0
QC Name            0
dtype: int64

Количество отрицательных значений по числовым столбцам:
Year                0
Month               0
Day                 0
Hour                0
NowCast Conc.     221
AQI               221
Raw Conc.        1055
dtype: int64

Сводная статистика для числовых столбцов:
               Year         Mo

In [4]:
def clean_data(data):
    data = data.replace(-999, pd.NA)
    data = data.infer_objects()

    data['NowCast Conc.'] = data['NowCast Conc.'].fillna(data['NowCast Conc.'].mean())
    data['AQI'] = data['AQI'].fillna(data['AQI'].mean())
    data['Raw Conc.'] = data['Raw Conc.'].fillna(data['Raw Conc.'].median())

    data['AQI Category'] = data['AQI Category'].fillna(data['AQI Category'].mode()[0])

    data['NowCast Conc.'] = data['NowCast Conc.'].apply(lambda x: x if x >= 0 else pd.NA)
    data['AQI'] = data['AQI'].apply(lambda x: x if x >= 0 else pd.NA)
    data['Raw Conc.'] = data['Raw Conc.'].apply(lambda x: x if x >= 0 else pd.NA)

    data['NowCast Conc.'] = data['NowCast Conc.'].fillna(data['NowCast Conc.'].mean())
    data['AQI'] = data['AQI'].fillna(data['AQI'].mean())
    data['Raw Conc.'] = data['Raw Conc.'].fillna(data['Raw Conc.'].median())

    return data

data = clean_data(data)

missing_values_after = data.isnull().sum()
print("\nПропущенные значения после очистки:")
print(missing_values_after)

print("\nПосле очистки:")
print(data.head())


Пропущенные значения после очистки:
Site             0
Parameter        0
Date (LT)        0
Year             0
Month            0
Day              0
Hour             0
NowCast Conc.    0
AQI              0
AQI Category     0
Raw Conc.        0
Conc. Unit       0
Duration         0
QC Name          0
dtype: int64

После очистки:
      Site          Parameter            Date (LT)  Year  Month  Day  Hour  \
0  Bishkek  PM2.5 - Principal  2019-02-06 06:00 AM  2019      2    6     6   
1  Bishkek  PM2.5 - Principal  2019-02-06 07:00 AM  2019      2    6     7   
2  Bishkek  PM2.5 - Principal  2019-02-06 08:00 AM  2019      2    6     8   
3  Bishkek  PM2.5 - Principal  2019-02-06 09:00 AM  2019      2    6     9   
4  Bishkek  PM2.5 - Principal  2019-02-06 10:00 AM  2019      2    6    10   

   NowCast Conc.        AQI AQI Category  Raw Conc. Conc. Unit Duration  \
0      31.834013  76.035676     Moderate       12.0      UG/M3     1 Hr   
1      17.000000  61.000000     Moderate       20

In [2]:
import pandas as pd

file_path = r"C:\Users\Les\Downloads\coursework\data\processed\processed_data.csv"

data = pd.read_csv(file_path)

print("Первые строки данных:")
print(data.head())

placeholder_values = [-999, "-999", "N/A", "NULL", "-", "", " "]
placeholder_summary = {}

for placeholder in placeholder_values:
    placeholder_summary[str(placeholder)] = (data == placeholder).sum().sum()

print("\nКоличество значений-заглушек:")
for key, value in placeholder_summary.items():
    print(f"{key}: {value}")

missing_values = data.isnull().sum().sum()
print(f"\nПропущенные значения (NaN): {missing_values}")


Первые строки данных:
      Site          Parameter            Date (LT)  Year  Month  Day  Hour  \
0  Bishkek  PM2.5 - Principal  2019-02-06 06:00:00  2019      2    6     6   
1  Bishkek  PM2.5 - Principal  2019-02-06 07:00:00  2019      2    6     7   
2  Bishkek  PM2.5 - Principal  2019-02-06 08:00:00  2019      2    6     8   
3  Bishkek  PM2.5 - Principal  2019-02-06 09:00:00  2019      2    6     9   
4  Bishkek  PM2.5 - Principal  2019-02-06 10:00:00  2019      2    6    10   

   NowCast Conc.        AQI AQI Category  Raw Conc. Conc. Unit Duration  \
0      31.834013  76.035676     Moderate       12.0      UG/M3     1 Hr   
1      17.000000  61.000000     Moderate       20.0      UG/M3     1 Hr   
2      19.100000  66.000000     Moderate       21.0      UG/M3     1 Hr   
3      22.400000  73.000000     Moderate       25.0      UG/M3     1 Hr   
4      30.900000  91.000000     Moderate       39.0      UG/M3     1 Hr   

  QC Name  
0   Valid  
1   Valid  
2   Valid  
3   Valid 

In [5]:
if data['Date (LT)'].dtype != 'datetime64[ns]':
    print("Столбец 'Date (LT)' не имеет формат datetime. Преобразуем его в datetime.")
    data['Date (LT)'] = pd.to_datetime(data['Date (LT)'], format='%Y-%m-%d %I:%M %p')

data['Year'] = data['Date (LT)'].dt.year
data['Month'] = data['Date (LT)'].dt.month
data['Day'] = data['Date (LT)'].dt.day
data['Hour'] = data['Date (LT)'].dt.hour

print("\nТипы данных после преобразования:")
print(data.dtypes)

print("\nПервые строки данных после преобразования:")
print(data.head())


Столбец 'Date (LT)' не имеет формат datetime. Преобразуем его в datetime.

Типы данных после преобразования:
Site                     object
Parameter                object
Date (LT)        datetime64[ns]
Year                      int32
Month                     int32
Day                       int32
Hour                      int32
NowCast Conc.           float64
AQI                     float64
AQI Category             object
Raw Conc.               float64
Conc. Unit               object
Duration                 object
QC Name                  object
dtype: object

Первые строки данных после преобразования:
      Site          Parameter           Date (LT)  Year  Month  Day  Hour  \
0  Bishkek  PM2.5 - Principal 2019-02-06 06:00:00  2019      2    6     6   
1  Bishkek  PM2.5 - Principal 2019-02-06 07:00:00  2019      2    6     7   
2  Bishkek  PM2.5 - Principal 2019-02-06 08:00:00  2019      2    6     8   
3  Bishkek  PM2.5 - Principal 2019-02-06 09:00:00  2019      2    6     9   


In [6]:
numerical_columns = ['NowCast Conc.', 'AQI', 'Raw Conc.']
print("\nТипы данных для числовых столбцов:")
print(data[numerical_columns].dtypes)

if 'Conc. Unit' in data.columns:
    print("\nУникальные значения в 'Conc. Unit':")
    print(data['Conc. Unit'].unique())


Типы данных для числовых столбцов:
NowCast Conc.    float64
AQI              float64
Raw Conc.        float64
dtype: object

Уникальные значения в 'Conc. Unit':
['UG/M3']


In [10]:
import os

processed_folder_path = r"C:\Users\Les\Downloads\coursework\data\processed"

os.makedirs(processed_folder_path, exist_ok=True)

processed_file_path = os.path.join(processed_folder_path, "processed_data.csv")

data.to_csv(processed_file_path, index=False)

print(f"\nОбработанные данные сохранены в файл: {processed_file_path}")



Обработанные данные сохранены в файл: C:\Users\Les\Downloads\coursework\data\processed\processed_data.csv
